In [2]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
import subprocess
from sklearn.utils import shuffle
import itertools
from sklearn.feature_selection import  f_classif

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

"""
    cross validation
"""
path="/home02/chenhuangrong/thaliana/"
inputname="thaliana_knn_RFH_4mer.csv"
outputname="thaliana_knn_RFH"
datapath =path+inputname
print "start"
train_data = pd.read_csv(datapath, header=None, index_col=None)
Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
Y.extend(Y2)
Y = np.array(Y)
F, pval = f_classif(train_data, Y)
idx = np.argsort(F)
selected_list_=idx[::-1]
bestACC=0
bestC=0
bestgamma=0
for select_num in xrange(1,376):
    train_data2=train_data.values
    selected_list_2=selected_list_[xrange(select_num)]
    X_train=pd.DataFrame(train_data2)
    X_train=X_train.iloc[:,selected_list_2]
    X = np.array(X_train)
    svc = svm.SVC()
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=8)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=8).mean()
    y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=8,method='predict_proba')
    predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    predict_save=np.array(predict_save).T
#     pd.DataFrame(predict_save).to_csv(path+name+'_predict.csv',header=None,index=False)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    if ACC>bestACC:
        bestACC=ACC
        bestgamma=gamma
        bestC=C
    print savedata
    print X.shape[1]
print bestACC
print bestC
print bestgamma
#     easy_excel.save("svm_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+name+'.xls')


start
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 394.0, 0.0, 394.0, 0.0, 394.0, 394.0]]]
1
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 394.0, 0.0, 394.0, 0.0, 394.0, 394.0]]]
2
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 394.0, 0.0, 394.0, 0.0, 394.0, 394.0]]]
3
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 394.0, 0.0, 394.0, 0.0, 394.0, 394.0]]]
4
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 394.0, 0.0, 394.0, 0.0, 394.0, 394.0]]]
5
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 394.0, 0.0, 394.0, 0.0, 394.0, 394.0]]]
6
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.